# Sort and commute

1. import hdf5 archives
2. decode hamiltonians from hdf5's
3. Sort paulis by size, group into commuting buckets, keep coefficients
    - Learn cirq/of package

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import cirq
import openfermion as of

import math
import os
from typing import Callable, Iterable, List, Set
#from tqdm.notebook import tqdm

1. Import hdf5 archives

In [2]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


def download_and_unzip(url, extract_to='.'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)

In [3]:
url="https://portal.nersc.gov/cfs/m888/dcamps/hamlib/chemistry/electronic/standard/"
molecules=['O2', 'B2', 'BeH', 'BH', 'CH', 'HF', 'C2', 'OH', 'N2', 'Li2', 'NaLi']


for molecule in molecules:
    if os.path.isfile('./'+molecule+'.hdf5') != True:
        download_and_unzip(url+molecule+'.hdf5.zip')


2. Decode hdf5 hamiltonians, done primarily using code from k-commute paper

In [4]:
import h5py


def read_openfermion_hdf5(fname_hdf5: str, key: str, optype=of.QubitOperator):
    """
    Read any openfermion operator object from HDF5 file at specified key.
    'optype' is the op class, can be of.QubitOperator or of.FermionOperator.
    """

    with h5py.File(fname_hdf5, 'r', libver='latest') as f:
        op = optype(f[key][()].decode("utf-8"))
    return op


def parse_through_hdf5(func):
    """
    Decorator function that iterates through an HDF5 file and performs
    the action specified by ‘ func ‘ on the internal and leaf nodes in the HDF5 file.
    """

    def wrapper (obj, path = '/', key = None) :
        if type(obj) in [h5py._hl.group.Group, h5py._hl.files.File]:
            for ky in obj.keys() :
                func(obj, path, key=ky, leaf = False)
                wrapper(obj = obj[ky], path = path + ky + ',', key = ky)
        elif type (obj) == h5py._hl.dataset.Dataset:
            func(obj, path, key = None, leaf = True)
    return wrapper


def get_hdf5_keys ( fname_hdf5 : str ) :
    """ Get a list of keys to all datasets stored in the HDF5 file .
    Args
    ----
    fname_hdf5 ( str ) : full path where HDF5 file is stored
    """

    all_keys = []
    @parse_through_hdf5
    def action(obj, path = '/', key = None, leaf = False):
        if leaf is True :
            all_keys.append(path)

    with h5py.File(fname_hdf5, 'r') as f:
        action(f['/'])
    return all_keys

In [5]:
molecule_to_read = 'O2'

hamiltonian = read_openfermion_hdf5(
    './'+molecule_to_read+'.hdf5',
    get_hdf5_keys('./'+molecule_to_read+'.hdf5')[0].rstrip(",")
)


Sort by size

In [6]:

hamiltonian = hamiltonian

sorted_h = sorted(hamiltonian.terms.items(), key=lambda x: abs(x[-1]), reverse=True)
#get back to of operators
sorted_hamiltonian = [of.QubitOperator(string[0],string[1]) for string in sorted_h]

openfermion.ops.operators.qubit_operator.QubitOperator

Group into commuting sets

In [7]:
def restrict_to(
    pauli: cirq.PauliString, qubits: Iterable[cirq.Qid]
) -> cirq.PauliString:
    """Returns the Pauli string restricted to the provided qubits.

    Arguments:
        pauli: A Pauli string.
        qubits: A set of 6 qubits.

    Returns:
        The provided Pauli string acting only on the provided qubits.
        Note: This could potentially be empty (identity).
    """
    return cirq.PauliString(p.on(q) for q, p in pauli.items() if q in qubits) #this is used in commutes()


def commutes(pauli1: cirq.PauliString, pauli2: cirq.PauliString, k: int = 1) -> bool:
    """Returns True if pauli1 k-commutes with pauli2, else False.

    Arguments:
        pauli1: A Pauli string.
        pauli2: A Pauli string.
        k: The integer k in k-commutativity.
    """
    all_qubits = sorted(set(pauli1.qubits).union(pauli2.qubits)) #checks depth of circuit
    if k >= len(all_qubits):
        return cirq.commutes(pauli1, pauli2) #so just does normal commuting

    blocks = [
        all_qubits[k * i : k * (i + 1)] for i in range(math.ceil(len(all_qubits) / k))
    ]
    for block in blocks:
        if not cirq.commutes(restrict_to(pauli1, block), restrict_to(pauli2, block)):
            return False
    return True

In [18]:

of.transforms.qubit_operator_to_pauli_sum(sorted_hamiltonian[2])

cirq.PauliSum(cirq.LinearDict({frozenset({(cirq.LineQubit(1), cirq.Z), (cirq.LineQubit(0), cirq.Z)}): (1.8741571908615127+0j)}))